In [ ]:
###########################################
# 0. Huggingface login

# READ token
!huggingface-cli login --token  # your code

In [ ]:
###########################################
# 1-1. 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
###########################################
# 1-2. 프로젝트 폴더 만들기

import os
os.makedirs('/content/drive/MyDrive/Sarcasm-LLM', exist_ok=True)

In [ ]:

%cd /content/drive/MyDrive/Sarcasm-LLM

In [ ]:
!pip install transformers datasets evaluate bleurt openai tqdm pandas

In [ ]:
!pip install evaluate

In [ ]:
##########################################
# 1-3. 모듈 다운

!pip install bitsandbytes
!pip install peft
!pip install accelerate
!pip install appdirs
!pip install loralib
!pip install black black[jupyter]
# !pip install datasets
!pip install fire
!pip install git+https://github.com/huggingface/peft
!pip install transformers
!pip install sentencepiece sentence_transformers
!pip install scipy numpy scikit-learn pandas
!pip install --upgrade datasets

In [ ]:
!git clone https://github.com/Beomi/ko-lm-evaluation-harness
%cd ./ko-lm-evaluation-harness
!pip install -e .
!pip install evaluate

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import evaluate
import openai
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [ ]:
###########################################
# 1-2. 프로젝트 폴더 만들기

import os
os.makedirs('/content/drive/MyDrive/Sarcasm-LLM', exist_ok=True)

In [ ]:

%cd /content/drive/MyDrive/Sarcasm-LLM

In [ ]:
import pandas as pd

# train
df_train1 = pd.read_csv('conversation_train.csv')
df_train2 = pd.read_csv('KoCoSa_train.csv', encoding='cp949')
df_train2 = df_train2.reset_index(drop=True)
df_train3 = pd.read_csv('review_train.csv')
df_train4 = pd.read_csv('comment_train.csv')

# test
df_test1 = pd.read_csv('conversation_test.csv')
df_test2 = pd.read_csv('KoCoSa_test.csv', encoding='cp949')
df_test2 = df_test2.reset_index(drop=True)
df_test3 = pd.read_csv('review_test.csv')
df_test4 = pd.read_csv('comment_test.csv')


# train/test 데이터프레임 합치기
df_train_combined = pd.concat([df_train1, df_train2, df_train3, df_train4], ignore_index=True)
df_test_combined = pd.concat([df_test1, df_test2, df_test3, df_test4], ignore_index=True)

data = {}

data['train']= df_train_combined
data['test']= df_test_combined


train_data = data['train']
test_data = data['test']

In [ ]:
from typing import Optional

# 새 sarcasm 프롬프트 템플릿
template = {
    "prompt_is_explanation": (
        "아래는 한국어 상황 'context'를 보고, 발화(말, 댓글 등)'response'에 대한 풍자 여부를 판별하는 예제입니다. "
        "풍자를 판별하고 이를 설명하는 답변을 작성해주세요.\n\n"
        "### 상황:\n{context}\n\n"
        "### 발화:\n{response}\n\n"
        "### 답변:\n"
    ),
    "prompt_no_explanation": (
        "아래는 발화에 대한 상황을 추측해보고, 발화(말, 댓글 등)'response'에 대한 풍자 여부를 판별하는 예제입니다. "
        "풍자를 판별하는 답변만 작성해주세요.\n\n"
        "### 발화:\n{response}\n\n"
        "### 답변:\n"
    ),
    "response_split": "### 답변:"
}

class Prompter(object):
    """
    context, response 를 입력으로 받고,
    optional label 과 optional explanation 을 target 출력으로 묶어
    prompt 를 생성합니다.
    """
    def __init__(self, verbose: bool = False):
        self.template = template
        self.verbose = verbose

    def generate_prompt(
        self,
        context: str,
        response: str,
        label: Optional[str] = None,
        explanation: Optional[str] = None
    ) -> str:
        # explanation 이 있으면 분류+설명 템플릿 사용
        if explanation is not None:
            prompt = self.template["prompt_is_explanation"].format(
                context=context,
                response=response
            )
            if label:
                prompt += label
            prompt += f"\n{explanation}"
        # explanation 이 없으면 분류만 템플릿 사용
        else:
            prompt = self.template["prompt_no_explanation"].format(
                response=response
            )
            if label:
                prompt += label

        if self.verbose:
            print("Generated prompt:\n", prompt)
        return prompt

    def get_response(self, output: str) -> str:
        """
        모델 출력에서 '### 답변:' 뒤에 나오는 실제 답변만 추출합니다.
        """
        parts = output.split(self.template["response_split"])
        return parts[-1].strip() if len(parts) > 1 else output.strip()

        # 사용 예시
prompter = Prompter(verbose=True)


In [ ]:
def tokenize(prompt: str, add_eos_token: bool = True):
    # (기존 토크나이저 그대로)
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        add_eos_token
        and result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    # labels 복사
    result["labels"] = result["input_ids"].copy()
    return result

def generate_and_tokenize_prompt(example, add_eos_token: bool = True):
    """
    example: {
      'context': str,
      'response': str,
      'label': 'Sarcasm' or 'Non-Sarcasm',
      'explanation': Optional[str]
    }
    """
    # 1) full prompt 생성
    full_prompt = prompter.generate_prompt(
        context=example["context"],
        response=example["response"],
        label=example.get("label"),
        explanation=example.get("explanation")
    )
    tokenized_full = tokenize(full_prompt, add_eos_token=add_eos_token)

    # 2) train_on_inputs=False 인 경우, 입력(prompt) 부분만 마스킹
    if not train_on_inputs:
        # prompt-only (label/explanation 없이) 길이를 알아냄
        prompt_only = prompter.generate_prompt(
            context=example["context"],
            response=example["response"]
        )
        tokenized_prompt = tokenize(prompt_only, add_eos_token=add_eos_token)
        prompt_len = len(tokenized_prompt["input_ids"])
        # EOS 토큰을 별도 마스킹했다면 길이에서 -1
        if add_eos_token:
            prompt_len -= 1
        # labels 의 앞 prompt_len 부분을 -100 으로
        tokenized_full["labels"] = ([-100] * prompt_len
                                    + tokenized_full["labels"][prompt_len:])

    return tokenized_full

# Base Model Zeroshot

In [ ]:
from tqdm import tqdm
import torch

# raw_test, tokenizer, model, device 는 이미 세팅된 상태라고 가정

correct_zs = 0
total_zs   = len(raw_test)

for i, ex in enumerate(tqdm(raw_test, desc="Zero-shot evaluation")):
    ctx, resp, true_lbl = ex["context"], ex["response"], ex["label"]

    # 1) 숫자 레이블 안내문구
    header = (
       "다음 대화의 마지막 발언이 풍자(Sarcasm)인지 아닌지 판단하세요.\n"
       "위 발언이 Sarcasm인지 Non-Sarcasm인지 '한 단어'로만 답하세요.\n"
       "답변은 숫자 1(=Sarcasm) 또는 0(=Non-Sarcasm) 중 하나의 토큰으로만 작성해주세요.\n\n"
    )
    # 2) Context + 발화까지 명시적으로 포함한 프롬프트
    eval_prompt = (
        header +
        f"### 문맥:\n{ctx}\n\n" +
        f"### 마지막 발언:\n{resp}\n\n" +
        "레이블: "
    )

    # 3) 모델 생성 (숫자 하나)
    toks = tokenizer(eval_prompt, return_tensors="pt").to(device)
    ln   = toks["input_ids"].shape[-1]
    out  = model.generate(
        **toks,
        max_new_tokens = 1,
        do_sample      = False,
        eos_token_id   = tokenizer.eos_token_id
    )
    gen_ids  = out[0][ln:]
    pred_num = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    pred_lbl = "Sarcasm" if pred_num == "1" else "Non-Sarcasm"

    # 4) 디버깅: context, true, pred 함께 출력 (처음 5개)
    if i < 5:
        # context가 너무 길면 앞부분만 자르려면 ctx[:100]처럼 조정 가능
        tqdm.write(
            f"#{i:02d}\n"
            f"문맥: {ctx}\n"
            f"발화: {resp}\n"
            f"실제: {true_lbl:<12}  →  pred_num: {pred_num:<2}  pred_label: {pred_lbl}\n"
            "—" * 40
        )

    if pred_lbl == true_lbl:
        correct_zs += 1

accuracy_zs = correct_zs / total_zs * 100
print(f"\nZero-shot classification accuracy: {accuracy_zs:.2f}%")


# FT Zero shot

In [ ]:
from tqdm import tqdm
import torch

# raw_test, tokenizer, model, device 는 이미 세팅된 상태라고 가정

correct_zs = 0
total_zs   = len(raw_test)

for i, ex in enumerate(tqdm(raw_test, desc="Zero-shot evaluation")):
    ctx, resp, true_lbl = ex["context"], ex["response"], ex["label"]

    # 1) 숫자 레이블 안내문구
    header = (
       "다음 대화의 마지막 발언이 풍자(Sarcasm)인지 아닌지 판단하세요.\n"
       "위 발언이 Sarcasm인지 Non-Sarcasm인지 '한 단어'로만 답하세요.\n"
       "답변은 숫자 1(=Sarcasm) 또는 0(=Non-Sarcasm) 중 하나의 토큰으로만 작성해주세요.\n\n"
    )
    # 2) Context + 발화까지 명시적으로 포함한 프롬프트
    eval_prompt = (
        header +
        f"### 문맥:\n{ctx}\n\n" +
        f"### 마지막 발언:\n{resp}\n\n" +
        "레이블: "
    )

    # 3) 모델 생성 (숫자 하나)
    toks = tokenizer(eval_prompt, return_tensors="pt").to(device)
    ln   = toks["input_ids"].shape[-1]
    out  = model.generate(
        **toks,
        max_new_tokens = 1,
        do_sample      = False,
        eos_token_id   = tokenizer.eos_token_id
    )
    gen_ids  = out[0][ln:]
    pred_num = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    pred_lbl = "Sarcasm" if pred_num == "1" else "Non-Sarcasm"

    # 4) 디버깅: context, true, pred 함께 출력 (처음 5개)
    if i < 5:
        # context가 너무 길면 앞부분만 자르려면 ctx[:100]처럼 조정 가능
        tqdm.write(
            f"#{i:02d}\n"
            f"문맥: {ctx}\n"
            f"발화: {resp}\n"
            f"실제: {true_lbl:<12}  →  pred_num: {pred_num:<2}  pred_label: {pred_lbl}\n"
            "—" * 40
        )

    if pred_lbl == true_lbl:
        correct_zs += 1

accuracy_zs = correct_zs / total_zs * 100
print(f"\nZero-shot classification accuracy: {accuracy_zs:.2f}%")


In [ ]:
!pip install git+https://github.com/google-research/bleurt.git

# FT GLEU

In [ ]:
# 0) 필수 라이브러리 임포트
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


# 2) 토크나이저·모델 로드
# Hugging Face 모델과 데이터셋 명칭
model_name = "myoo-oong/Korean-Sarcasm-LLM-llama-3.2-Korean-Bllossom-3B-CustomData"
# 토크나이저 및 모델 로드 (로컬에 모델이 없으면 처음 실행 시 다운로드)
tokenizer = AutoTokenizer.from_pretrained(model_name)




model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# 🧩 1. train/test DataFrame → HuggingFace Dataset 변환
train_df = data["train"].reset_index(drop=True)
test_df = data["test"].reset_index(drop=True)

# 🧩 2. train-validation 분할
df_train_combined, df_val = train_test_split(train_df, test_size=0.2, random_state=42)

# 🧩 3. HuggingFace Dataset 변환
train_dataset = Dataset.from_pandas(df_train_combined.reset_index(drop=True))
val_dataset = Dataset.from_pandas(df_val.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df)

# 🧩 4. DatasetDict 구성 (선택 사항)
data_hf = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})


# 1) 원본(raw) train/test 데이터셋 불러오기
raw_train    = data_hf["train"]
raw_test     = data_hf["test"]

# 3) 디바이스 세팅 (GPU 우선)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 4) Prompter 클래스 인스턴스 생성
#    (이미 정의해 두신 `Prompter` 코드를 그대로 사용)
prompter = Prompter(verbose=False)

In [ ]:
!pip install git+https://github.com/google-research/bleurt.git


In [ ]:
from tqdm import tqdm
import evaluate
import openai
import pandas as pd
import time

# 평가 대상 샘플
eval_samples = raw_test.select(range(810))
generated_texts = []
reference_texts = []

# 생성
for sample in tqdm(eval_samples, desc="Generating outputs"):
    context   = sample['context']
    response  = sample['response']
    header = (
        "아래 대화에 대해, 마지막 발언이 풍자인지 여부를 판별하고, "
        "한국어로 그 이유를 자세히 설명해 주세요.\n"
        "만약, Sarcasm, Non-Sarcasm에 대해 중의적 표현(ambiguous expression)이 등장할 경우,"
        "반드시 전체 문맥(context)와 화자의 전후 대화 흐름을 고려하여 풍자 여부를 판별하십시오.\n"
        "특히 긍정적인 말처럼 보이지만 반어적 의미를 내포할 수 있는 표현은 비꼼이나 불평의 의도를 추론하여 풍자로 간주하는 경향을 가지되,"
        "명백한 근거가 없는 경우 신중하게 비풍자로 분류합니다.\n\n"
    )
    gen_prompt = (
        header +
        f"### 문맥:\n{context}\n\n"
        f"### 발화:\n{response}\n\n"
        "### 답변:\n"
    )
    inputs = tokenizer(gen_prompt, return_tensors="pt").to(device)
    input_len = inputs["input_ids"].shape[-1]
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id
    )
    gen_ids  = outputs[0][input_len:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    generated_texts.append(gen_text)

    label       = sample.get('label', '') or ''
    explanation = sample.get('explanation', '') or ''
    label       = label.strip()
    explanation = explanation.strip()
    ref_text = f"{label}\n{explanation}" if explanation else label
    reference_texts.append(ref_text)

# BLEURT 평가를 위한 정제
clean_preds = []
clean_refs = []
clean_indexes = []
for idx, (pred, ref) in enumerate(zip(generated_texts, reference_texts)):
    if isinstance(pred, str) and isinstance(ref, str) and pred.strip() and ref.strip():
        clean_preds.append(pred)
        clean_refs.append(ref)
        clean_indexes.append(idx)

# BLEURT
print("▶ BLEURT 평가 시작...")
bleurt = evaluate.load("bleurt", config_name="bleurt-base-128")
bleurt_scores = bleurt.compute(predictions=clean_preds, references=clean_refs)["scores"]
df_bleurt = pd.DataFrame({
    "Index": clean_indexes,
    "BLEURT": bleurt_scores,
    "Generated": [generated_texts[i] for i in clean_indexes],
    "Reference": [reference_texts[i] for i in clean_indexes]
})
print(df_bleurt["BLEURT"].describe())

# BLEURT 전체 결과 CSV 저장
df_bleurt.to_csv("bleurt_scores_custom.csv", index=False)

# GPT-4 설정
openai.api_key = "" # ← 본인 API Key 입력

def evaluate_with_gpt4(conversation, reference, model_output):
    system_prompt = (
        "당신은 풍자 검출 평가 전문가입니다. 주어진 대화, 정답 레이블+설명, 모델 출력을 보고 "
        "모델이 생성한 설명이 얼마나 정확한지 0~10 점수로 평가하세요. 숫자로만 답하세요."
    )
    user_prompt = (
        f"[대화]\n{conversation}\n\n"
        f"[정답]\n{reference}\n\n"
        f"[모델 출력]\n{model_output}\n\n"
        "위 모델 출력의 품질을 0에서 10 사이 점수로 평가해주세요."
    )
    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        score_str = response.choices[0].message.content.strip()
        return float(score_str)
    except Exception as e:
        print(f"[ERROR] GPT-4 평가 실패: {e}")
        return None

# GPT-4 평가
gpt4_scores = []
errors = []
print("▶ GPT-4 평가 시작...")

for i in tqdm(range(len(eval_samples))):
    conv = f"문맥: {eval_samples[i]['context']}\n발화: {eval_samples[i]['response']}"
    ref  = reference_texts[i]
    out  = generated_texts[i]
    score = evaluate_with_gpt4(conv, ref, out)
    gpt4_scores.append(score)
    if score is None:
        errors.append(i)
    time.sleep(1.1)

# 결과 병합 및 저장
df_gpt4 = pd.DataFrame({
    "Sample": list(range(len(eval_samples))),
    "GPT-4 Score": gpt4_scores,
    "Generated": generated_texts,
    "Reference": reference_texts
})
df_gpt4.to_csv("gpt4_sarcasm_scores_custom.csv", index=False)

# ▶ 결과 통계
print("\n▶ GPT-4 평가 요약:")
print(df_gpt4["GPT-4 Score"].describe())

# ▶ BLEURT 최고/최저 예시 출력
best_idx = df_bleurt["BLEURT"].idxmax()
worst_idx = df_bleurt["BLEURT"].idxmin()

print("\n🟢 BLEURT 최고 점수 예시")
print("점수       :", df_bleurt.loc[best_idx, "BLEURT"])
print("예측 답변 :", df_bleurt.loc[best_idx, "Generated"])
print("참조 답변 :", df_bleurt.loc[best_idx, "Reference"])

print("\n🔴 BLEURT 최저 점수 예시")
print("점수       :", df_bleurt.loc[worst_idx, "BLEURT"])
print("예측 답변 :", df_bleurt.loc[worst_idx, "Generated"])
print("참조 답변 :", df_bleurt.loc[worst_idx, "Reference"])

# 저장
df_bleurt.loc[[best_idx]].to_csv("bleurt_best_example_custom.csv", index=False)
df_bleurt.loc[[worst_idx]].to_csv("bleurt_worst_example_custom.csv", index=False)

# GPT-4 평가 실패 인덱스 저장
if errors:
    with open("gpt4_errors_custom.txt", "w") as f:
        for i in errors:
            f.write(f"{i}\n")
    print(f"\n⚠️ GPT-4 평가 실패 {len(errors)}건: gpt4_errors_custom.txt에 저장됨.")


# Base model GLEU

In [ ]:
# 0) 필수 라이브러리 임포트
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 1) 원본(raw) train/test 데이터셋 불러오기
raw_train    = data_hf["train"]
raw_test     = data_hf["test"]

# 2) 토크나이저·모델 로드
# Hugging Face 모델과 데이터셋 명칭
model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
# 토크나이저 및 모델 로드 (로컬에 모델이 없으면 처음 실행 시 다운로드)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

# 3) 디바이스 세팅 (GPU 우선)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 4) Prompter 클래스 인스턴스 생성
#    (이미 정의해 두신 `Prompter` 코드를 그대로 사용)
prompter = Prompter(verbose=False)

In [ ]:
from tqdm import tqdm
import evaluate
import openai
import pandas as pd
import time

# 평가 대상 샘플
eval_samples = raw_test.select(range(810))
generated_texts = []
reference_texts = []

# 생성
for sample in tqdm(eval_samples, desc="Generating outputs"):
    context   = sample['context']
    response  = sample['response']
    header = (
        "아래 대화에 대해, 마지막 발언이 풍자인지 여부를 판별하고, "
        "한국어로 그 이유를 자세히 설명해 주세요.\n"
        "만약, Sarcasm, Non-Sarcasm에 대해 중의적 표현(ambiguous expression)이 등장할 경우,"
        "반드시 전체 문맥(context)와 화자의 전후 대화 흐름을 고려하여 풍자 여부를 판별하십시오.\n"
        "특히 긍정적인 말처럼 보이지만 반어적 의미를 내포할 수 있는 표현은 비꼼이나 불평의 의도를 추론하여 풍자로 간주하는 경향을 가지되,"
        "명백한 근거가 없는 경우 신중하게 비풍자로 분류합니다.\n\n"
    )
    gen_prompt = (
        header +
        f"### 문맥:\n{context}\n\n"
        f"### 발화:\n{response}\n\n"
        "### 답변:\n"
    )
    inputs = tokenizer(gen_prompt, return_tensors="pt").to(device)
    input_len = inputs["input_ids"].shape[-1]
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id
    )
    gen_ids  = outputs[0][input_len:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    generated_texts.append(gen_text)

    label       = sample.get('label', '') or ''
    explanation = sample.get('explanation', '') or ''
    label       = label.strip()
    explanation = explanation.strip()
    ref_text = f"{label}\n{explanation}" if explanation else label
    reference_texts.append(ref_text)

# BLEURT 평가를 위한 정제
clean_preds = []
clean_refs = []
clean_indexes = []
for idx, (pred, ref) in enumerate(zip(generated_texts, reference_texts)):
    if isinstance(pred, str) and isinstance(ref, str) and pred.strip() and ref.strip():
        clean_preds.append(pred)
        clean_refs.append(ref)
        clean_indexes.append(idx)

# BLEURT
print("▶ BLEURT 평가 시작...")
bleurt = evaluate.load("bleurt", config_name="bleurt-base-128")
bleurt_scores = bleurt.compute(predictions=clean_preds, references=clean_refs)["scores"]
df_bleurt = pd.DataFrame({
    "Index": clean_indexes,
    "BLEURT": bleurt_scores,
    "Generated": [generated_texts[i] for i in clean_indexes],
    "Reference": [reference_texts[i] for i in clean_indexes]
})
print(df_bleurt["BLEURT"].describe())
# BLEURT 전체 결과 CSV 저장
df_bleurt.to_csv("bleurt_scores_base.csv", index=False)



# ▶ BLEURT 최고/최저 예시 출력
best_idx = df_bleurt["BLEURT"].idxmax()
worst_idx = df_bleurt["BLEURT"].idxmin()

print("\n🟢 BLEURT 최고 점수 예시")
print("점수       :", df_bleurt.loc[best_idx, "BLEURT"])
print("예측 답변 :", df_bleurt.loc[best_idx, "Generated"])
print("참조 답변 :", df_bleurt.loc[best_idx, "Reference"])

print("\n🔴 BLEURT 최저 점수 예시")
print("점수       :", df_bleurt.loc[worst_idx, "BLEURT"])
print("예측 답변 :", df_bleurt.loc[worst_idx, "Generated"])
print("참조 답변 :", df_bleurt.loc[worst_idx, "Reference"])

# 저장
df_bleurt.loc[[best_idx]].to_csv("bleurt_best_example_base.csv", index=False)
df_bleurt.loc[[worst_idx]].to_csv("bleurt_worst_example_base.csv", index=False)

# GPT-4 평가 실패 인덱스 저장
if errors:
    with open("gpt4_errors_base.txt", "w") as f:
        for i in errors:
            f.write(f"{i}\n")
    print(f"\n⚠️ GPT-4 평가 실패 {len(errors)}건: gpt4_errors_base.txt에 저장됨.")
